## Begin Exploring Data and Building Features

The data disaggregates variables multiple ways, male and female, white, African-American, Hispanic, Asian-American, IEP/504, etc. and covers 34 topical areas including chronic absenteeism of both students and teachers, subjects, curriculum focus, sports participation, harassment and bullying, teacher certification, discipline, budgets, and more. In all situations, the features used the summed the male and female numbers and did not consider disaggregated groups. Even so, there were more than 170 columns of total male, female or other variables. The combined data set was more than 1,800 columns wide and had data on more 21,000 high schools.

Basic data cleaning involved deleting unused columns, totaling male and female number to get a total for the school. Often when schools did not have the data available or immediately available they were given the option of adding a negative number as an indicator to explain the missing data. Those negative numbers were converted to NANs.

The variables are all created in the notebook. The exploratory data and visualization are in other notebooks, especially, Simple_numbers, Big_and_Small EDA, and Large_Schools_First_Models.

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [50]:
#combo is the combined graduation and features data sets
combo = pd.read_csv('/Users/flatironschool/Absenteeism_Project/data/processed/grad_data.csv')

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,11,13,15,17,19,21,23,25,51,52,53,58,59,1856) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
combo.head()

,Unnamed: 0,STNAM,FIPST,LEAID_x,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,MAM_COHORT_1516,...,SCH_JJTYPE,SCH_JJSYDAYS,SCH_JJHOURS,SCH_JJPART_LT15,SCH_JJPART_15T30,SCH_JJPART_31T90,SCH_JJPART_91T180,SCH_JJPART_OV180,districtID,IDSCH
0,0,ALABAMA,1,100005,Albertville City,10000500871,Albertville High Sch,296,92,2,...,-9,-9,-9,-9,-9,-9,-9,-9,100005,871
1,1,ALABAMA,1,100006,Marshall County,10000600872,Asbury Sch,67,GE95,2,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,872
2,2,ALABAMA,1,100006,Marshall County,10000600878,Douglas High Sch,153,85-89,.,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,878
3,3,ALABAMA,1,100006,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,.,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,883
4,4,ALABAMA,1,100006,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,2,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,1585


In [52]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21867 entries, 0 to 21866
Columns: 1866 entries, Unnamed: 0 to IDSCH
dtypes: float64(32), int64(1748), object(86)
memory usage: 311.3+ MB


The OCR (features) data set disaggregates the data by ethnicity, gender, 
504/IEP/disability status, English language learner, etc. This project 
uses the total for each variable, not the disaggregated. The cell below
returned totals to begin the aggregation process

In [53]:
total_cols = [col for col in combo.columns if 'TOT' in col]
print(total_cols)

['TOT_PSENR_M', 'TOT_PSENR_F', 'TOT_ENR_M', 'TOT_ENR_F', 'TOT_LEPENR_M', 'TOT_LEPENR_F', 'TOT_LEPPROGENR_M', 'TOT_LEPPROGENR_F', 'TOT_IDEAENR_M', 'TOT_IDEAENR_F', 'TOT_504ENR_M', 'TOT_504ENR_F', 'TOT_GTENR_M', 'TOT_GTENR_F', 'TOT_DUAL_M', 'TOT_DUAL_F', 'TOT_ALGENR_G08_M', 'TOT_ALGENR_G08_F', 'TOT_ALGENR_GS0910_M', 'TOT_ALGENR_GS0910_F', 'TOT_ALGENR_GS1112_M', 'TOT_ALGENR_GS1112_F', 'TOT_ALGPASS_G08_M', 'TOT_ALGPASS_G08_F', 'TOT_ALGPASS_GS0910_M', 'TOT_ALGPASS_GS0910_F', 'TOT_ALGPASS_GS1112_M', 'TOT_ALGPASS_GS1112_F', 'TOT_GEOM_M', 'TOT_GEOM_F', 'TOT_MATHENR_ALG2_M', 'TOT_MATHENR_ALG2_F', 'TOT_MATHENR_CALC_M', 'TOT_MATHENR_CALC_F', 'TOT_MATHENR_ADVM_M', 'TOT_MATHENR_ADVM_F', 'TOT_SCIENR_BIOL_M', 'TOT_SCIENR_BIOL_F', 'TOT_SCIENR_CHEM_M', 'TOT_SCIENR_CHEM_F', 'TOT_SCIENR_PHYS_M', 'TOT_SCIENR_PHYS_F', 'TOT_SSCLASSES_ALGG', 'TOT_SSCLASSES_OTHM', 'TOT_SSCLASSES_SCI', 'TOT_SSCLASSES_ENGL', 'TOT_SSCLASSES_OTHA', 'TOT_APENR_M', 'TOT_APENR_F', 'TOT_APMATHENR_M', 'TOT_APMATHENR_F', 'TOT_APSCIENR_

In [54]:
len(total_cols)

177

In [55]:
#drop columns of disaggregated graduation rates
combo.drop(['FIPST', 'LEAID_x', 'MAM_COHORT_1516', 'MAM_RATE_1516',
                    'MAS_COHORT_1516', 'MAS_RATE_1516', 'MBL_COHORT_1516', 
                    'MBL_RATE_1516','MHI_COHORT_1516', 'MHI_RATE_1516', 
                    'MTR_COHORT_1516','MTR_RATE_1516', 'MWH_COHORT_1516',
                    'MWH_RATE_1516', 'CWD_COHORT_1516','CWD_RATE_1516',
                    'ECD_COHORT_1516','ECD_RATE_1516','LEP_COHORT_1516',
                    'LEP_RATE_1516','DATE_CUR'], axis=1, inplace=True)


In [56]:
combo.head()

,Unnamed: 0,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,LEAID_y,...,SCH_JJTYPE,SCH_JJSYDAYS,SCH_JJHOURS,SCH_JJPART_LT15,SCH_JJPART_15T30,SCH_JJPART_31T90,SCH_JJPART_91T180,SCH_JJPART_OV180,districtID,IDSCH
0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,100005,...,-9,-9,-9,-9,-9,-9,-9,-9,100005,871
1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,100006,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,872
2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,100006,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,878
3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,100006,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,883
4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,100006,...,-9,-9,-9,-9,-9,-9,-9,-9,100006,1585


## Replace All Negative Values with NANs
If a category did not apply to the school (for example, the school does not offer AP), they plan on getting the data later, small cell value, missing data, system error or some other reason, the variables were assigned a negative number. These have been replaced with NANs.

In [57]:
combo.replace([-2, -5, -6, -7, -8, -9], np.nan, inplace=True)

In [59]:
combo['TOT_ALGPASS_GS0910_F'].describe()

count    16229.000000
mean        62.693265
std         65.245564
min          4.000000
25%         16.000000
50%         38.000000
75%         92.000000
max       1470.000000
Name: TOT_ALGPASS_GS0910_F, dtype: float64

## Create New Features, Delete Disaggregated Variables

In [60]:
#create total enrollment feature and drop the rest of the 
#disaggregated enrollment features
combo['total_enrollment'] = combo['TOT_ENR_M'] + combo['TOT_ENR_F']

In [61]:
combo.drop(combo.columns.to_series()['SCH_PSENR_NONIDEA_A3':'SCH_ENR_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_ENR_LEP_M':'SCH_504ENR_LEP_F'], axis=1, inplace=True)

In [62]:
combo.drop(combo.columns.to_series()['SCH_GRADE_PS':'SCH_UGDETAIL_HS'], axis=1, inplace=True)

In [63]:
#create new feature combining advanced classes: AP, IB, Duel Enrollment 
#(enrolling in a class at the local community college)

combo['total_ap_ib_de'] = combo['TOT_APENR_M'] + combo['TOT_APENR_F'] + combo['TOT_IBENR_M'] + combo['TOT_IBENR_F'] + combo['TOT_DUAL_M'] + combo['TOT_DUAL_F']
combo['ap_ib_de_rate'] = combo['total_ap_ib_de']/combo['total_enrollment']

In [64]:
combo.drop(['SCH_DUALENR_LEP_M', 'SCH_DUALENR_LEP_F', 'SCH_DUALENR_IDEA_M', 'SCH_DUALENR_IDEA_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_APENR_IND':'SCH_APENR_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_APENR_LEP_M':'SCH_APPASS_NONE_IDEA_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_IBENR_IND':'SCH_IBENR_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_IBENR_LEP_M':'SCH_IBENR_IDEA_F'], axis=1, inplace=True)

In [65]:
combo.drop(combo.columns.to_series()['SCH_GTENR_LEP_M':'SCH_DUALENR_TR_F'], axis=1, inplace=True)

In [66]:
combo.drop(combo.columns.to_series()['SCH_GT_IND':'SCH_GTENR_TR_F'], axis=1, inplace=True)

In [67]:
#create new feature of number and rate of students taking SAT or ACT for
#college admission. Deleting disaggregated features
combo['total_sat_act'] = combo['TOT_SATACT_M'] + combo['TOT_SATACT_F']
combo['sat_act_rate'] = combo['total_sat_act']/combo['total_enrollment']

In [68]:
combo.drop(combo.columns.to_series()['SCH_SATACT_HI_M':'SCH_SATACT_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_SATACT_LEP_M':'SCH_SATACT_IDEA_F'], axis=1, inplace=True)

In [69]:
#create new feature of number and rate of students passing Algebra for all grades
combo['total_pass_algebra'] = combo['TOT_ALGPASS_GS0910_M'] + combo['TOT_ALGPASS_GS0910_F'] + combo['TOT_ALGPASS_GS1112_M'] + combo['TOT_ALGPASS_GS1112_F']
combo['pass_algebra_rate'] = combo['total_pass_algebra']/combo['total_enrollment']

In [70]:
#create new feature of students enrolled in geometry
combo['total_geometry'] = combo['TOT_GEOM_M'] + combo['TOT_GEOM_F']
combo['geometry_rate'] = combo['total_geometry']/combo['total_enrollment']

In [71]:
#create new feature of students enrolled in algebra 2
combo['total_algebra2'] = combo['TOT_MATHENR_ALG2_M'] + combo['TOT_MATHENR_ALG2_F']
combo['algebra2_rate'] = combo['total_algebra2']/combo['total_enrollment']

In [72]:
#create new feature of students enrolled in calculus
combo['total_calc'] = combo['TOT_MATHENR_CALC_M'] + combo['TOT_MATHENR_CALC_F']
combo['calc_rate'] = combo['total_calc']/combo['total_enrollment']

In [73]:
#create new feature of number and rate of students that are chronically 
#absent, missing 15 or more school days in a school year. All absences, 
#excused and unexcused are included. Deleting disaggreated features.
combo['total_chronic_absent'] = combo['TOT_ABSENT_M'] + combo['TOT_ABSENT_F']
combo['chronic_absent_rate'] = combo['total_chronic_absent']/combo['total_enrollment']

In [74]:
combo.drop(combo.columns.to_series()['SCH_ABSENT_HI_M':'SCH_ABSENT_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_ABSENT_LEP_M':'SCH_ABSENT_IDEA_F'], axis=1, inplace=True)

In [75]:
#create new feature of number and rate of students participating in sports
#delete disaggregated variables
combo['sports_part'] = combo['SCH_SSPART_M'] + combo['SCH_SSPART_F']
combo['sports_rate'] = combo['sports_part']/combo['total_enrollment']

In [76]:
combo.drop(combo.columns.to_series()['SCH_SSATHLETICS_IND':'TOT_SSTEAMS'], axis=1, inplace=True)

In [77]:
#create new feature related to the number of days of school lost to suspensions.
#Using this feature will penalize longer suspensions.
combo['total_suspension_days'] = combo['TOT_DAYSMISSED_F'] + combo['TOT_DAYSMISSED_F']
combo['suspensed_day_rate'] = combo['total_suspension_days']/combo['total_enrollment']

In [78]:
combo.drop(combo.columns.to_series()['SCH_PSDISC_SINGOOS_HI_M':'SCH_DAYSMISSED_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_DAYSMISSED_LEP_M':'SCH_DAYSMISSED_IDEA_F'], axis=1, inplace=True)

In [79]:
#Create new feature related to number and rate of alleged harassment or bullying 
#on the basis of gender, race, disability, sexual orientation, or religion.
combo['harassed'] = combo['SCH_HBALLEGATIONS_SEX'] + combo['SCH_HBALLEGATIONS_RAC'] + combo['SCH_HBALLEGATIONS_DIS'] + combo['SCH_HBALLEGATIONS_ORI'] + combo['SCH_HBALLEGATIONS_REL']
combo['harassed_rate'] = combo['harassed']/combo['total_enrollment']

In [80]:
combo.drop(combo.columns.to_series()['SCH_HBREPORTED_SEX_HI_M':'SCH_HBDISCIPLINED_DIS_504_F'], axis=1, inplace=True)

In [81]:
#Create new feature related to non-personnel expenditures related to 
#activities funded by Federal, State and local funds. Delete other features 
combo['activities_funds_rate'] = (combo['SCH_NPE_WOFED'] + combo['SCH_NPE_WFED'])/combo['total_enrollment']

In [82]:
combo.drop(['SCH_SAL_TOTPERS_WOFED', 'SCH_FTE_TEACH_WOFED', 'SCH_SAL_TEACH_WOFED'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_FTE_AID_WOFED':'SCH_SAL_TOTPERS_WFED'], axis=1, inplace=True)

## Features related to teachers and staff

In [83]:
#rate of non-certified teachers
combo['non_cert_rate'] = combo['SCH_FTETEACH_NOTCERT']/combo['SCH_FTETEACH_TOT']

In [84]:
#rate of school counselors per student
combo['counselor_rate']= combo['SCH_FTECOUNSELORS']/combo['total_enrollment']

In [85]:
#rate of chronically absent teachers
combo['absent_teacher_rate'] = combo['SCH_FTETEACH_ABSENT']/combo['SCH_FTETEACH_TOT']

In [86]:
combo.drop(['SCH_FTETEACH_CERT', 'SCH_FTETEACH_FY', 'SCH_FTETEACH_SY', 'SCH_TEACHERS_CURR_TOT',
           'SCH_TEACHERS_PREV_TOT', 'SCH_FTESECURITY_LEO', 'SCH_FTESECURITY_GUA', 'SCH_FTESERVICES_NUR',
           'SCH_FTESERVICES_PSY','SCH_FTESERVICES_SOC'], axis=1, inplace=True)

## Delete Disaggregated Variables

Drop sections on retention, corporal discipline, explusions, transfers, 
referrals, arrests, offenses, restraint, seclusion, disabilites, and juvenile justice 

In [87]:
combo.drop(combo.columns.to_series()['SCH_CREDITRECOVERY_IND':'SCH_ALGENR_GS0910_TR_F'], axis=1, inplace=True)

In [88]:
combo.drop(combo.columns.to_series()['SCH_ALGENR_GS0910_LEP_M':'SCH_ALGENR_GS1112_TR_F'],axis=1, inplace= True)
combo.drop(combo.columns.to_series()['SCH_ALGENR_GS1112_LEP_M':'SCH_ALGPASS_GS0910_TR_F'],axis=1, inplace=True)

In [89]:
combo.drop(combo.columns.to_series()['SCH_ALGPASS_GS1112_LEP_M':'SCH_MATHENR_GEOM_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_MATHENR_GEOM_LEP_M':'SCH_MATHENR_ALG2_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_MATHENR_ALG2_LEP_M':'SCH_MATHENR_CALC_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_MATHENR_CALC_LEP_M':'SCH_MATHENR_ADVM_TR_F'], axis=1, inplace=True)

In [90]:
combo.drop(combo.columns.to_series()['SCH_MATHENR_ADVM_LEP_M':'SCH_SCIENR_BIOL_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_SCIENR_BIOL_LEP_M':'SCH_SCIENR_CHEM_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_SCIENR_CHEM_LEP_M':'SCH_SCIENR_PHYS_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_SCIENR_PHYS_LEP_M':'TOT_SSCLASSES_OTHA'], axis=1, inplace=True)

In [91]:
combo.drop(combo.columns.to_series()['SCH_ALGPASS_GS0910_LEP_M':'SCH_ALGPASS_GS1112_TR_F'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_RS_NONIDEA_MECH_HI_M':'SCH_RSINSTANCES_SECL_504'],axis=1, inplace=True)

In [92]:
combo.drop(combo.columns.to_series()['SCH_CORPINSTANCES_IND':'SCH_CORPINSTANCES_WODIS'], axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_PSDISC_EXP_HI_M':'SCH_OFFENSE_POSSWX'],axis=1, inplace=True)

In [93]:
combo.drop(combo.columns.to_series()['SCH_RET_KG_IND':'SCH_RET_G12_504_F'],axis=1, inplace=True)
combo.drop(combo.columns.to_series()['SCH_JJTYPE':'SCH_JJPART_OV180'], axis=1, inplace=True)

In [94]:
combo.drop('SCH_CORPINSTANCES_WDIS', axis=1, inplace=True)

In [95]:
columns = combo.columns
for col in columns:
    print(col)

Unnamed: 0
STNAM
LEANM
NCESSCH
SCHNAM
ALL_COHORT_1516
ALL_RATE_1516
LEA_STATE
LEA_STATE_NAME
LEAID_y
LEA_NAME
SCHID
SCH_NAME
COMBOKEY
JJ
SCH_STATUS_SPED
SCH_STATUS_MAGNET
SCH_STATUS_CHARTER
SCH_STATUS_ALT
SCH_MAGNETDETAIL
SCH_ALTFOCUS
TOT_ENR_M
TOT_ENR_F
TOT_GTENR_M
TOT_GTENR_F
TOT_DUAL_M
TOT_DUAL_F
TOT_ALGENR_GS0910_M
TOT_ALGENR_GS0910_F
TOT_ALGENR_GS1112_M
TOT_ALGENR_GS1112_F
TOT_ALGPASS_GS0910_M
TOT_ALGPASS_GS0910_F
TOT_ALGPASS_GS1112_M
TOT_ALGPASS_GS1112_F
TOT_GEOM_M
TOT_GEOM_F
TOT_MATHENR_ALG2_M
TOT_MATHENR_ALG2_F
TOT_MATHENR_CALC_M
TOT_MATHENR_CALC_F
TOT_MATHENR_ADVM_M
TOT_MATHENR_ADVM_F
TOT_SCIENR_BIOL_M
TOT_SCIENR_BIOL_F
TOT_SCIENR_CHEM_M
TOT_SCIENR_CHEM_F
TOT_SCIENR_PHYS_M
TOT_SCIENR_PHYS_F
TOT_APENR_M
TOT_APENR_F
TOT_IBENR_M
TOT_IBENR_F
TOT_SATACT_M
TOT_SATACT_F
TOT_DAYSMISSED_M
TOT_DAYSMISSED_F
SCH_HBALLEGATIONS_SEX
SCH_HBALLEGATIONS_RAC
SCH_HBALLEGATIONS_DIS
SCH_HBALLEGATIONS_ORI
SCH_HBALLEGATIONS_REL
TOT_ABSENT_M
TOT_ABSENT_F
SCH_SSPART_M
SCH_SSPART_F
TOT_SSPART
SCH_NPE_WO

In [96]:
combo.describe()

,Unnamed: 0,NCESSCH,ALL_COHORT_1516,LEAID_y,SCHID,COMBOKEY,TOT_ENR_M,TOT_ENR_F,TOT_GTENR_M,TOT_GTENR_F,...,sports_part,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate
count,21867.000000,2.186700e+04,21867.000000,2.186700e+04,21867.000000,2.186700e+04,21866.000000,21866.000000,10924.000000,10924.000000,...,15082.000000,15082.000000,21826.000000,21826.000000,21732.000000,21732.000000,2.082300e+04,21506.000000,21863.000000,21506.000000
mean,10933.000000,2.781234e+11,161.376686,2.781234e+06,2843.349248,2.781234e+11,360.312357,344.848349,53.040828,56.683449,...,364.775759,0.460135,162.131953,0.260529,1.612875,0.003623,5.149183e+03,0.033132,0.005183,0.243690
std,6312.603504,1.598186e+11,180.009443,1.598186e+06,3524.581030,1.598186e+11,379.231611,371.949762,82.680920,93.011643,...,316.553964,0.254176,376.150184,0.761871,8.996756,0.029979,6.228301e+04,0.104288,0.053906,0.213044
min,0.000000,1.000050e+10,1.000000,1.000050e+05,1.000000,1.000050e+10,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,5466.500000,1.300120e+11,32.000000,1.300120e+06,685.000000,1.300120e+11,85.000000,76.000000,7.000000,7.000000,...,121.000000,0.282259,0.000000,0.000000,0.000000,0.000000,5.709937e+02,0.000000,0.002135,0.064103
50%,10933.000000,2.718570e+11,90.000000,2.718570e+06,1615.000000,2.718570e+11,218.000000,209.000000,23.000000,23.000000,...,277.000000,0.423059,32.000000,0.069052,0.000000,0.000000,1.321879e+03,0.000000,0.003125,0.216216
75%,16399.500000,4.101725e+11,245.000000,4.101725e+06,3670.000000,4.101725e+11,534.000000,513.000000,64.000000,66.000000,...,535.000000,0.600897,156.000000,0.227467,1.000000,0.000970,3.482353e+03,0.012373,0.004587,0.361882
max,21866.000000,5.606240e+11,4713.000000,5.606240e+06,90469.000000,5.606240e+11,6713.000000,7733.000000,1008.000000,1170.000000,...,3700.000000,3.846154,9606.000000,34.444444,760.000000,2.473684,5.882562e+06,1.000000,6.809524,1.348315


In [97]:
combo.to_csv('combo_cleaned.csv')